In [1]:
import json
import io
import folium
import seaborn as sns
from folium.features import DivIcon
from PIL import Image

In [2]:
with open('../data/matched-results-1500.json', 'r') as f:
    match_results = json.load(f)
print(len(match_results))

1500


In [3]:
pal = sns.color_palette('pastel', n_colors=15).as_hex()

In [4]:
initial_location = [41.1579, -8.6291]  # Example coordinates for Porto, Portugal
fmap = folium.Map(location=initial_location, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(fmap)

for i in range(15):
    gps_points = match_results[i]['POLYLINE']
    gps_points = [p[::-1] for p in gps_points]

    matched_geometry = match_results[i]['MATCHED_RESULTS']['Matched_geometry']
    mgeom = [i[::-1] for i in matched_geometry] 
    
    folium.PolyLine(mgeom, color=pal[i], weight=8, opacity=1).add_to(fmap) 
    
    folium.CircleMarker(location=mgeom[0], radius=8, color=pal[i], fill_color=pal[i], fill_opacity=1).add_to(fmap)
    folium.CircleMarker(location=mgeom[-1], radius=8, color=pal[i], fill_color=pal[i], fill_opacity=1).add_to(fmap)
    
    folium.map.Marker(
        mgeom[0],
        icon=DivIcon(icon_anchor=(5, 10), html=f'<div style="font-size: 10pt">S{i+1}</div>'),
    ).add_to(fmap)    
    folium.map.Marker(
        mgeom[-1],
        icon=DivIcon(icon_anchor=(5, 10), html='<div style="font-size: 10pt">E</div>')
    ).add_to(fmap)
    
    #for point in gps_points:
    #    folium.CircleMarker(location=point, radius=1, color='black').add_to(fmap)
    
fmap.save('../data/matched-15-routes-folium.html')

img_data = fmap._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('../data/matched-15-routes-folium.png')

fmap

In [5]:
for i in range(15):
    gps_points = match_results[i]['POLYLINE']
    gps_points = [p[::-1] for p in gps_points]

    latitudes, longitudes = zip(*gps_points)
    centroid = [(max(latitudes)+min(latitudes))/2, (max(longitudes)+min(longitudes))/2]

    matched_geometry = match_results[i]['MATCHED_RESULTS']['Matched_geometry']
    mgeom = [i[::-1] for i in matched_geometry] 

    fmap = folium.Map(location=centroid, zoom_start=15)
    folium.TileLayer('cartodbpositron').add_to(fmap)

    folium.PolyLine(mgeom, color=pal[i], weight=8, opacity=1).add_to(fmap)
    
    folium.CircleMarker(location=mgeom[0], radius=8, color=pal[i], fill_color=pal[i], fill_opacity=1).add_to(fmap)
    folium.CircleMarker(location=mgeom[-1], radius=8, color=pal[i], fill_color=pal[i], fill_opacity=1).add_to(fmap)
    
    folium.map.Marker(
        mgeom[0],
        icon=DivIcon(icon_anchor=(5, 10), html=f'<div style="font-size: 10pt">S{i+1}</div>'),
    ).add_to(fmap)    
    folium.map.Marker(
        mgeom[-1],
        icon=DivIcon(icon_anchor=(5, 10), html='<div style="font-size: 10pt">E</div>')
    ).add_to(fmap)

    for j, point in enumerate(gps_points):
        folium.CircleMarker(location=point, radius=1, color='black').add_to(fmap)
        folium.map.Marker(
            location=point,
            icon=DivIcon(icon_anchor=(5, 20), html=f'<div style="font-size: 8pt">%s</div>' % str(j+1)),
        ).add_to(fmap)

    # Save each map as an HTML file
    fmap.save(f'../data/matched-route-{i}-folium.html')

    img_data = fmap._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    img.save(f'../data/matched-route-{i}-folium.png')
    
fmap

In [6]:
i = 3

gps_points = match_results[i]['POLYLINE']
gps_points = [p[::-1] for p in gps_points]
print(len(gps_points))

latitudes, longitudes = zip(*gps_points)
centroid = [(max(latitudes)+min(latitudes))/2, (max(longitudes)+min(longitudes))/2]
print(centroid)

matched_geometry = match_results[i]['MATCHED_RESULTS']['Matched_geometry']
mgeom = [i[::-1] for i in matched_geometry] 
print(len(mgeom))

43
[41.1472575, -8.5927545]
423


In [7]:
_gps_points = gps_points[:18]
_mgeom = mgeom[:244]

fmap = folium.Map(location=centroid, zoom_start=15)
folium.TileLayer('cartodbpositron').add_to(fmap)

folium.PolyLine(_mgeom, color=pal[i], weight=8, opacity=1).add_to(fmap)

folium.CircleMarker(location=_mgeom[0], radius=8, color=pal[i], fill_color=pal[i], fill_opacity=1).add_to(fmap)
folium.map.Marker(
    _mgeom[0],
    icon=DivIcon(icon_anchor=(5, 10), html=f'<div style="font-size: 10pt">S{i+1}</div>'),
).add_to(fmap)    

for j, point in enumerate(_gps_points, 0):
    folium.CircleMarker(location=point, radius=1, color='black').add_to(fmap)
    folium.map.Marker(
        location=point,
        icon=DivIcon(icon_anchor=(5, 20), html=f'<div style="font-size: 10pt">%s</div>' % str(j+1)),
    ).add_to(fmap)

fmap.save(f'../data/matched-route-{i}-folium-1.html')

img_data = fmap._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save(f'../data/matched-route-{i}-folium-1.png')
    
fmap


In [8]:
_gps_points = gps_points[17:]
_mgeom = mgeom[242:]

fmap = folium.Map(location=centroid, zoom_start=15)
folium.TileLayer('cartodbpositron').add_to(fmap)

folium.PolyLine(_mgeom, color=pal[i], weight=8, opacity=1).add_to(fmap)

folium.CircleMarker(location=_mgeom[-1], radius=8, color=pal[i], fill_color=pal[i], fill_opacity=1).add_to(fmap)   
folium.map.Marker(
    _mgeom[-1],
    icon=DivIcon(icon_anchor=(5, 10), html='<div style="font-size: 10pt">E</div>')
).add_to(fmap)

for j, point in enumerate(_gps_points, 17):
    folium.CircleMarker(location=point, radius=1, color='black').add_to(fmap)
    folium.map.Marker(
        location=point,
        icon=DivIcon(icon_anchor=(5, 20), html=f'<div style="font-size: 10pt">%s</div>' % str(j+1)),
    ).add_to(fmap)
    
fmap.save(f'../data/matched-route-{i}-folium-2.html')

img_data = fmap._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save(f'../data/matched-route-{i}-folium-2.png')

fmap